In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_counts = application_df.loc[:, 'APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# skipping visualization

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_type_counts[app_type_counts < 200].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_value_counts = application_df['CLASSIFICATION'].value_counts()
classification_value_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# skipping visualization

In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_value_counts[classification_value_counts < 800].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Generate our categorical variable lists (19.4.2)
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Optimization Attempt 1

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 25)                775       
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 6,751
Trainable params: 6,751
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
804/804 [==============================] - 1s 653us/step - loss: 23223.0781 - accuracy: 0.4916
Epoch 2/50
804/804 [==============================] - 1s 683us/step - loss: 5546.0171 - accuracy: 0.4864
Epoch 3/50
804/804 [==============================] - 1s 654us/step - loss: 881.0314 - accuracy: 0.5174
Epoch 4/50
804/804 [==============================] - 1s 673us/step - loss: 0.7080 - accuracy: 0.5318
Epoch 5/50
804/804 [==============================] - 1s 660us/step - loss: 0.6914 - accuracy: 0.5318
Epoch 6/50
804/804 [==============================] - 1s 673us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 7/50
804/804 [==============================] - 1s 657us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 8/50
804/804 [==============================] - 1s 665us/step - loss: 0.6912 - accuracy: 0.5318
Epoch 9/50
804/804 [==============================] - 1s 657us/step - loss: 0.6911 - accuracy: 0.5318
Epoch 10/50
804/804 [==============================] - 1s 653us/step - lo

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.7035 - accuracy: 0.5349 - 180ms/epoch - 673us/step
Loss: 0.7035254836082458, Accuracy: 0.5349271297454834


In [18]:
# Export attempt 2 model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_Attempt1.h5")

# Optimization Attempt 2

In [19]:
# Define the model - using X_train_scaled
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 1s 611us/step - loss: 0.5807 - accuracy: 0.7170
Epoch 2/50
804/804 [==============================] - 0s 617us/step - loss: 0.5587 - accuracy: 0.7280
Epoch 3/50
804/804 [==============================] - 0s 617us/step - loss: 0.5551 - accuracy: 0.7289
Epoch 4/50
804/804 [==============================] - 0s 620us/step - loss: 0.5526 - accuracy: 0.7306
Epoch 5/50
804/804 [==============================] - 0s 613us/step - loss: 0.5513 - accuracy: 0.7291
Epoch 6/50
804/804 [==============================] - 0s 615us/step - loss: 0.5498 - accuracy: 0.7293
Epoch 7/50
804/804 [==============================] - 0s 615us/step - loss: 0.5489 - accuracy: 0.7304
Epoch 8/50
804/804 [==============================] - 0s 607us/step - loss: 0.5481 - accuracy: 0.7320
Epoch 9/50
804/804 [==============================] - 0s 608us/step - loss: 0.5477 - accuracy: 0.7303
Epoch 10/50
804/804 [==============================] - 0s 611us/step - loss: 0.547

In [20]:
# Export attempt 2 model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_Attempt2.h5")

# Optimization Attempt 3

In [21]:
# Define the model - using X_train_scaled
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 75
hidden_nodes_layer2 = 50
hidden_nodes_layer2 = 25   # add another hidden layer

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 1s 670us/step - loss: 0.5694 - accuracy: 0.7203
Epoch 2/50
804/804 [==============================] - 1s 679us/step - loss: 0.5546 - accuracy: 0.7290
Epoch 3/50
804/804 [==============================] - 1s 683us/step - loss: 0.5515 - accuracy: 0.7306
Epoch 4/50
804/804 [==============================] - 1s 677us/step - loss: 0.5498 - accuracy: 0.7317
Epoch 5/50
804/804 [==============================] - 1s 674us/step - loss: 0.5488 - accuracy: 0.7310
Epoch 6/50
804/804 [==============================] - 1s 668us/step - loss: 0.5485 - accuracy: 0.7309
Epoch 7/50
804/804 [==============================] - 1s 666us/step - loss: 0.5474 - accuracy: 0.7321
Epoch 8/50
804/804 [==============================] - 1s 694us/step - loss: 0.5465 - accuracy: 0.7332
Epoch 9/50
804/804 [==============================] - 1s 671us/step - loss: 0.5466 - accuracy: 0.7332
Epoch 10/50
804/804 [==============================] - 1s 666us/step - loss: 0.545

In [22]:
# Export attempt 3 model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_Attempt3.h5")

# Optimization Attempt 4

In [23]:
# Define the model - using X_train_scaled and increase the nodes
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 1000
hidden_nodes_layer2 = 500

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5716 - accuracy: 0.7200
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5572 - accuracy: 0.7298
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5536 - accuracy: 0.7307
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5521 - accuracy: 0.7298
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7320
Epoch 6/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5501 - accuracy: 0.7317
Epoch 7/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5482 - accuracy: 0.7332
Epoch 8/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5481 - accuracy: 0.7333
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5466 - accuracy: 0.7328
Epoch 10/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accuracy: 0.7334

In [24]:
# Export attempt 4 model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization_Attempt4.h5")